# Recreate `gputest` environment (safe, recommended steps)

Run the following commands in a Windows cmd terminal (one block at a time). These commands:
- remove the old environment
- create a fresh environment
- install NumPy with conda (so we avoid DLL/pip conflicts)
- install PyTorch wheels built for CUDA 11.3 via PyTorch's index
- install the rest of the Python packages via pip (requirements.txt)

1) Remove old env and create a new one:

```cmd
conda deactivate
conda env remove -n gputest -y
conda create -n gputest python=3.10 -y
conda activate gputest
```

2) Install NumPy (conda):

```cmd
conda install numpy=1.24.3 -y
```

3) Install CUDA toolkit (conda) - optional but recommended:

```cmd
conda install cudatoolkit=11.3 -c conda-forge -y
```

4) Install PyTorch (pip) with CUDA 11.3 wheels from PyTorch's index:

```cmd
pip install --extra-index-url https://download.pytorch.org/whl/cu113 \
  torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1
```

5) Install the rest of the requirements (note: `numpy` is intentionally removed from requirements.txt):

```cmd
pip install -r requirements.txt
```

6) Restart the Jupyter kernel and select the `gputest` environment.

# Environment Setup with requirements.txt
Follow these steps to set up your environment:

1. First, remove the existing environment:
```bash
conda deactivate
conda env remove -n gputest -y
```

2. Create a new environment with Python 3.10:
```bash
conda create -n gputest python=3.10 -y
conda activate gputest
```

3. Install PyTorch dependencies:
```bash
conda install pytorch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 cudatoolkit=11.3 -c pytorch -y
```

4. Install other requirements:
```bash
pip install -r requirements.txt
```

After running these commands, restart the kernel and select the new 'gputest' environment.

In [1]:
import numpy as np
import cv2
from ultralytics import YOLO
import easyocr
import re
from collections import defaultdict, deque

ModuleNotFoundError: No module named 'cv2'

In [13]:
model = YOLO("license_plate_best.pt") # finetuned weights and biases .pt means pretrained
reader = easyocr.Reader(['en'], gpu=True)

# Regex pattern for license plate validation 2 letters, 2 digits, 3 letters (e.g., AB12CDE)
plate_pattern = re.compile(r'^[A-Z]{2}[0-9]{2}[A-Z]{3}$')
# Regex pattern for Algerian license plates: 5 or 6 digits, followed by 3 digits, followed by 2 digits
algerian_pattern = re.compile(r'^[0-9]{5,6}[0-9]{3}[0-9]{2}$')

In [14]:
def correct_plate_format(ocr_text):
    mapping_num_to_char = {'0': 'O', '1': 'I', '2': 'Z', '5': 'S', '8': 'B'}
    mapping_char_to_num = {v: k for k, v in mapping_num_to_char.items()}
    
    ocr_text = ocr_text.upper().replace(" ", "")
    if len(ocr_text) != 7:
        return None # Invalid length for standard plate
    corrected = []
    for i, char in enumerate(ocr_text):
        if i < 2 or i >= 4:  # Letter positions
            if char in mapping_num_to_char:
                corrected.append(mapping_num_to_char[char])
            elif char.isalpha():
                corrected.append(char)
            else:
                return None # Invalid character for letter position
        else:  # Digit positions
            if char in mapping_char_to_num:
                corrected.append(mapping_char_to_num[char])
            elif char.isdigit():
                corrected.append(char)
            else:
                return None # Invalid character for digit position
    return ''.join(corrected)

In [15]:
def recognize_plate(plate_crop):
    if plate_crop is None or plate_crop.size == 0:
        return ""
    
    #preprocess for OCR
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    plate_resized = cv2.resize(thresh, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    try:
        ocr_result = reader.readtext(plate_resized, detail=0,allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        if len(ocr_result) > 0:
            candidate = correct_plate_format(ocr_result[0])
            if candidate and plate_pattern.match(candidate):
                return candidate
    except:
        pass
    return ""

In [16]:
plate_history = defaultdict(lambda: deque(maxlen=10)) # Store last 10 readings for each plate ID
plate_final_counts = {}  # Store final confirmed plate for each detected plate ID
def get_box_id(x1,y1,x2,y2):
    return f"{int(x1/10)}_{int(y1/10)}_{int(x2/10)}_{int(y2/10)}"

def get_stable_plate(box_id,new_text):
    if new_text:
        plate_history[box_id].append(new_text)
        most_common = max(set(plate_history[box_id]), key=plate_history[box_id].count)
        plate_final_counts[box_id] = most_common
    return plate_final_counts.get(box_id, "")


In [23]:
input_video_path = "gettyimages-807748644-640_adpp.mp4"
output_video_path = "output_with_license_plates.mp4"

cap = cv2.VideoCapture(input_video_path) # if input_video_path is 0, it uses webcam
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(3)), int(cap.get(4))))

CONF_THRESH = 0.3

In [24]:
# Make sure numpy is properly imported and working
import numpy as np
import cv2
from ultralytics import YOLO

# Verify numpy is working
print("NumPy version:", np.__version__)

# Initialize video capture and output
input_video_path = "gettyimages-807748644-640_adpp.mp4"
output_video_path = "output_with_license_plates.mp4"

cap = cv2.VideoCapture(input_video_path) # if input_video_path is 0, it uses webcam
if not cap.isOpened():
    raise ValueError("Could not open video file")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Set confidence threshold for detections
CONF_THRESH = 0.3

NumPy version: 1.24.3


In [27]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame,verbose=False)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = float(box.conf.cpu().numpy()[0])
            if conf < CONF_THRESH:
                continue
            
            x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])
            plate_crop = frame[y1:y2, x1:x2]
            
            text = recognize_plate(plate_crop)
            
            box_id = get_box_id(x1,y1,x2,y2)
            stable_text = get_stable_plate(box_id, text)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            
            # Overlay zoomed-in plate above the detected plate
            if plate_crop.size > 0:
                overlay_h, overlay_w = 150, 400
                plate_resized = cv2.resize(plate_crop, (overlay_w, overlay_h))
                
                oy1 = max(0, y1 - overlay_h - 40)
                ox1 = x1
                oy2,ox2 = oy1 + overlay_h, ox1 + overlay_w
                
                if oy2 <= frame.shape[0] and ox2 <= frame.shape[1]:
                    frame[oy1:oy2, ox1:ox2] = plate_resized
                    
                    if stable_text:
                        cv2.putText(frame, stable_text, (ox1, oy1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 6)
                        cv2.putText(frame, stable_text, (ox1, oy1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255,255), 3)
    out.write(frame)
    cv2.imshow("License Plate Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Output saved to", output_video_path)

# Real-Time License Plate Detection and Recognition
This code will:
1. Process video input (from file or camera)
2. Detect license plates using YOLO model
3. Recognize text using EasyOCR
4. Display results in real-time
5. Save processed video with detections (optional)

In [21]:
def process_video(input_path=0, output_path=None, confidence_threshold=0.25):
    """
    Process video for license plate detection and recognition
    Args:
        input_path: Path to video file or camera index (default 0 for webcam)
        output_path: Path to save processed video (optional)
        confidence_threshold: Confidence threshold for YOLO detections
    """
    # Initialize video capture
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video source")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Initialize video writer if output path is specified
    writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    # Dictionary to store recognized plates and their counts
    plate_counts = defaultdict(int)
    # Queue to store recent detections for tracking
    recent_plates = deque(maxlen=30)  # Store last 30 frames worth of detections
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run YOLO detection
            results = model(frame)[0]
            
            # Process each detection
            for det in results.boxes.data.tolist():
                x1, y1, x2, y2, conf, _ = det
                
                if conf < confidence_threshold:
                    continue
                
                # Extract license plate region
                plate_roi = frame[int(y1):int(y2), int(x1):int(x2)]
                if plate_roi.size == 0:
                    continue
                
                # Recognize text
                ocr_results = reader.readtext(plate_roi)
                
                for (bbox, text, ocr_conf) in ocr_results:
                    # Try to correct and validate the plate format
                    corrected_text = correct_plate_format(text)
                    if corrected_text:
                        plate_counts[corrected_text] += 1
                        recent_plates.append(corrected_text)
                        
                        # Draw bounding box
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                        # Add text
                        cv2.putText(frame, corrected_text, (int(x1), int(y1)-10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Display most frequent recent detections
            y_pos = 30
            for plate, count in sorted(plate_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
                text = f"{plate}: {count}"
                cv2.putText(frame, text, (10, y_pos),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                y_pos += 30
            
            # Show frame
            cv2.imshow('License Plate Detection', frame)
            
            # Write frame if output path specified
            if writer:
                writer.write(frame)
            
            # Break on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        # Clean up
        cap.release()
        if writer:
            writer.release()
        cv2.destroyAllWindows()
        
        # Print final results
        print("\nDetected License Plates:")
        for plate, count in sorted(plate_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"{plate}: {count} detections")

# Usage Examples
Below are examples of how to use the video processing function:
1. For webcam input
2. For video file input
3. For video file input with saving output

In [22]:
# Example 1: Use webcam
# process_video()  # Uncomment to run


# Example 2: Process a video file
process_video("C:\\Users\\Zinou\\OneDrive\\Desktop\\3cs\\projects\\slm2\\06_License_Plate_Recognition_CV\\gettyimages-807748644-640_adpp.mp4")  # Uncomment and replace with your video path

# Example 3: Process a video file and save the output
# process_video("path_to_your_video.mp4", "output_video.mp4")  # Uncomment and replace with your paths

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1268: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
